In [2]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

2.6.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AutomatedQuestionGeneration/Topic_Bifurcated_SQUAD1.csv")
df.head()

,index,question,context,text,answer_start,c_id,given_context,reference_context
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,singing and dancing,207.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,2003,526.0,0,"Born and raised in Houston, Texas, she perform...",Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Houston, Texas",166.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,late 1990s,276.0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"Born and raised in Houston, Texas, she perform..."


In [4]:
df = df[:1000]




# Load data set




In [5]:
reference_context = df["reference_context"].apply(lambda x: x).tolist()
given_context = df["given_context"].apply(lambda x: x).tolist()
target_ques = df["question"].apply(lambda x: x).tolist()

*   Clean the sentences by removing special characters.
*   Add a start and end token to each sentence.
*   Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
*   Pad each sentence to a maximum length.

In [6]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",","¿")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  # remove extra space
  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
en_sentence = u"Beyonce's last record was called Lemonade"
sp_sentence = u"Beyonce was born in 1950"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))
print(preprocess_sentence(sp_sentence).encode("UTF-8"))

<start> beyonce s last record was called lemonade <end>
<start> beyonce was born in <end>
b'<start> beyonce was born in <end>'


In [ ]:
reference_context[:1], given_context[:1]

(['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy". '],
 ["Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. "])

In [8]:
def create_dataset_squad(context_param, given_param, target_ques):
  reference_context = []
  given_context = []
  question = []
  for c,g,q in zip(context_param,given_param, target_ques):
    reference_context.append(preprocess_sentence(str(c)))
    given_context.append(preprocess_sentence(str(g)))
    question.append(preprocess_sentence(q))
  return tuple(reference_context), tuple(given_context), tuple(question)

r, g, q = create_dataset_squad(reference_context, given_context, target_ques)
print(r[-1])
print(g[-1])
print(q[-1])
print(len(r), len(q))

<start> his fellow student at the warsaw conservatory , julian fontana , had originally tried unsuccessfully to establish himself in england albert grzyma a , who in paris became a wealthy financier and society figure , often acted as chopin s adviser and gradually began to fill the role of elder brother in his life . fontana was to become , in the words of micha owski and samson , chopin s general factotum and copyist . <end>
<start> two polish friends in paris were also to play important roles in chopin s life there . <end>
<start> who is stated as a jack of all trades in service to frederic ? <end>
1000 1000


In [9]:
# Tokenize the sentence into list of words(integers) and pad the sequence to the same length
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [10]:
def load_dataset(reference, given, target):
  # creating cleaned input, output pairs
  #targ_lang, inp_lang = create_dataset(path, num_examples)

  reference_tensor, reference_tokenizer = tokenize(reference)
  given_tensor, given_tokenizer = tokenize(given)
  target_tensor, targ_lang_tokenizer = tokenize(target)

  return reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_lang_tokenizer

In [11]:
# Try experimenting with the size of that dataset
#num_examples = 30000
# input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(c,q)
reference_tensor, given_tensor, target_tensor, reference_tokenizer, given_tokenizer, targ_tokenizer = load_dataset(r, g, q)

# Calculate max_length of the target tensors
max_length_targ, max_length_reference, max_length_given = target_tensor.shape[1], reference_tensor.shape[1], given_tensor.shape[1]
print(max_length_targ, max_length_reference, max_length_given)

30 256 136


In [ ]:
# train = zip(reference_tensor, given_tensor)
# train = pd.DataFrame(data=train, index=None)
# train.head()

In [12]:

# Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(train, target_tensor, test_size=0.2)
reference_tensor_train, reference_tensor_val, given_tensor_train, given_tensor_val, target_tensor_train, target_tensor_val = train_test_split(reference_tensor, given_tensor, target_tensor)

# Show length
# print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
# print(input_tensor_train[1])
# print()
# print(target_tensor_train[1])

In [13]:
# input_tensor_train
# reference_tensor_train = input_tensor_train[0].values
# given_tensor_train = input_tensor_train[1].values
# 
# reference_tensor_val = input_tensor_val[0].values
# given_tensor_val = input_tensor_val[1].values

In [14]:
given_tensor_train.shape, reference_tensor_train.shape

((750, 136), (750, 256))

In [15]:
type(given_tensor_train[0])

numpy.ndarray

In [ ]:
# np.expand_dims(given_tensor_train, axis=1).shape

In [18]:
# Configuration 
BUFFER_SIZE = len(reference_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(reference_tensor_train)//BATCH_SIZE
steps_per_epoch_val = len(reference_tensor_val)//BATCH_SIZE
embedding_dim = 256  # for word embedding
units = 1024  # dimensionality of the output space of RNN
vocab_inp_size = len(given_tokenizer.word_index) + len(reference_tokenizer.word_index) + 1
vocab_tar_size = len(targ_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_train, given_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
validation_dataset = tf.data.Dataset.from_tensor_slices((reference_tensor_val, given_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE, drop_remainder=True)

example_reference_batch, example_given_batch, example_target_batch = next(iter(dataset))
example_reference_batch.shape, example_given_batch.shape, example_target_batch.shape

(TensorShape([64, 256]), TensorShape([64, 136]), TensorShape([64, 30]))

# Seq2seq model: Two Encoders and DecoderWithDualAttention

In [61]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.LSTM = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,  # Whether to return the last output in the output sequence, or the full sequence. 
                                   return_state=True,  # Whether to return the last state in addition to the output.
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, hidden_h, hidden_c = self.LSTM(x, initial_state = hidden)
    return output, [hidden_h,hidden_c]

  def initialize_hidden_state(self):
    init_state = [tf.zeros([self.batch_sz, self.enc_units]) for i in range(2)]
    return init_state

In [65]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden= encoder(example_reference_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))
print ('Encoder Carry state shape: (batch size, units) {}'.format(sample_hidden[1].shape))

Encoder output shape: (batch size, sequence length, units) (64, 256, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)
Encoder Carry state shape: (batch size, units) (64, 1024)


In [95]:
sample_hidden

[<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[-0.00146165,  0.0086659 , -0.0076584 , ..., -0.00085536,
          0.01640438,  0.00312099],
        [-0.00146166,  0.00866589, -0.00765838, ..., -0.00085536,
          0.01640437,  0.00312101],
        [-0.00146173,  0.00866571, -0.00765818, ..., -0.00085545,
          0.01640431,  0.00312116],
        ...,
        [-0.00146165,  0.0086659 , -0.0076584 , ..., -0.00085536,
          0.01640438,  0.00312099],
        [-0.00146165,  0.0086659 , -0.0076584 , ..., -0.00085536,
          0.01640438,  0.00312099],
        [-0.00146165,  0.0086659 , -0.0076584 , ..., -0.00085536,
          0.01640438,  0.003121  ]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[-0.00294325,  0.0172829 , -0.01545583, ..., -0.00171862,
          0.0329176 ,  0.00623879],
        [-0.00294326,  0.01728288, -0.01545578, ..., -0.00171861,
          0.03291759,  0.00623883],
        [-0.00294341,  0.01728252, -0.01545538,

In [94]:
[tf.zeros([1, 2]) for i in range(2)]

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>]

In [93]:
[tf.zeros((1,2)),tf.zeros((1,2))]

[<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>]

In [30]:
%debug

> /usr/local/lib/python3.7/dist-packages/keras/layers/recurrent.py(619)_validate_state_spec()
    617 
    618     if len(flat_cell_state_sizes) != len(flat_state_specs):
--> 619       raise validation_error
    620     for cell_state_spec, cell_state_size in zip(flat_state_specs,
    621                                                 flat_cell_state_sizes):

ipdb> cell.get_initial_state(inputs)
*** NameError: name 'cell' is not defined
ipdb> state_spec
*** NameError: name 'state_spec' is not defined
ipdb> `cell.state_size`
*** SyntaxError: invalid syntax
ipdb> encoder.cell.state_size
*** NameError: name 'encoder' is not defined
ipdb> exit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



# Additive attention

![alt text](https://i.ibb.co/BqDYNP1/additive.jpg)

In [148]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)

    # print("Query : ,",query)
    
    query_with_time_axis = tf.expand_dims(query, 1)
    # print("Query with time axis: ,",query_with_time_axis.shape)
    # query_with_time_axis = query
    # values = query_with_time_axis
    # print("Values: ,",values.shape)
    # print(values.shape)
    # print(query_with_time_axis.shape)

    # (64, 358, 1024) ->values.shape
    # (64, 1, 1024) -> query_with_time_axis.shape

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(query_with_time_axis)))
    # print("W1(Values): ", self.W1(values).shape)
    # print("W2(Query_with_time_axis): ", self.W2(query_with_time_axis).shape)
    # print("Score: ", score.shape)
    # (64, 358, 1024) -> self.W1(values).shape
    # (64, 1, 1024) -> self.W2(query_with_time_axis).shape
    # (64, 358, 1) -> score.shape

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    # print(context_vector.shape)
    # (64, 358, 1024)

    context_vector = tf.reduce_sum(context_vector, axis=1)
    # print(context_vector.shape)
    # (64, 1024)
    return context_vector, attention_weights

In [153]:
class DecoderWithDualAttention(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, attention_layer1 = None, attention_layer2 = None):
    super(DecoderWithDualAttention, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.LSTM = tf.keras.layers.LSTM(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention1 = attention_layer1
    self.attention2 = attention_layer2

  def call(self, x, hidden1, hidden2, enc_output1, enc_output2):
    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    # print("X shape before concat: ",x.shape)
    y = x
    attention_weights1 = None   
    
    # Attention 1 is used to associate the target word embedding with the reference context embedding (attended)
    if self.attention1:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector1, attention_weights1 = self.attention1(hidden1[0], enc_output1)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      x = tf.concat([tf.expand_dims(context_vector1, 1), x], axis=-1)
      # x = tf.concat([context_vector1, x], axis=-1)
      # print("X shape after concat: ",x.shape)
      
    
    attention_weights2 = None
    
    # Attention 2 is used to associate the target word embedding with the given context embedding (attended)
    if self.attention2:
      # enc_output shape == (batch_size, max_length, hidden_size)
      context_vector2, attention_weights2 = self.attention2(hidden2[0], enc_output2)
      # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
      y = tf.concat([tf.expand_dims(context_vector2, 1), y], axis=-1)
      # y = tf.concat([context_vector2, y], axis=-1)

      

    # passing the concatenated vector to the LSTM
    output1, state_h1, state_c1 = self.LSTM(x, initial_state = hidden1)
    output2, state_h2, state_c2= self.LSTM(y, initial_state = hidden2)

    # output shape == (batch_size * 1, hidden_size)
    output1 = tf.reshape(output1, (-1, output1.shape[2]))
    output2 = tf.reshape(output2, (-1, output2.shape[2]))

    output = tf.concat([output1, output2], axis = 1)

    # output shape == (batch_size, vocab)
    final_output = self.fc(output)

    return final_output, [state_h1,state_c1], [state_h2,state_c2], attention_weights1, attention_weights2

In [115]:
y = tf.zeros((64,1,256))

In [113]:
c_vector = tf.expand_dims(tf.zeros((64,1024)),1)

In [118]:
tf.concat([c_vector, y], axis=-1).shape

TensorShape([64, 1, 1280])

In [48]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) 
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [49]:
print(loss_object([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))
print(loss_function([1,2],[[0,0.6,0.3,0.1],[0,0.6,0.3,0.1]]))

tf.Tensor([1.063386  1.3633859], shape=(2,), dtype=float32)
tf.Tensor(1.2133859, shape=(), dtype=float32)


In [149]:
optimizer = tf.keras.optimizers.Adam()

def get_train_step_func():

  @tf.function
  def train_step(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder):
    loss = 0

    with tf.GradientTape() as tape: # for automatic differentiation
      enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
      enc_output2, enc_hidden2 = encoder2(aux, enc_hidden2)

      # Initialising decoder hidden states with the corresponding hidden states of the encoder (one for given context encoder and the other for reference context encoder)
      dec_hidden1 = enc_hidden1
      dec_hidden2 = enc_hidden2
      # enc_hidden -> 

      # Check whether we should concatenate dec_hidden and dec_hidden2 or keep it separate
      # dec_hidden2 = enc_hidden2

      dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
        # print("Iteration: ",t)
        # passing enc_output to the decoder
        # print("EncOutput1 before: ",enc_output1)
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_output1, enc_output2)
        # print("EncOutput1 after: ",enc_output1)
        
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder1.trainable_variables + encoder2.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss
    
  return train_step

In [151]:
def caculate_validation_loss(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder):
  loss = 0
  enc_output1, enc_hidden1 = encoder1(inp, enc_hidden1)
  enc_output2, enc_hidden2 = encoder2(aux, enc_hidden1)
  dec_hidden1 = enc_hidden1
  dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

  # Teacher forcing - feeding the target as the next input
  # for t in range(1, targ.shape[1]):
  #   predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
  #   loss += loss_function(targ[:, t], predictions)
  #   dec_input = tf.expand_dims(targ[:, t], 1)

  # loss = loss / int(targ.shape[1])
  # return loss

  for t in range(1, targ.shape[1]):
        # passing enc_output to the decoder
        predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_output1, enc_output2)
        # x, y, hidden1, hidden2, enc_output1, enc_output2

        loss += loss_function(targ[:, t], predictions)

        # using teacher forcing
        dec_input = tf.expand_dims(targ[:, t], 1)

  loss = loss / int(targ.shape[1])
  return loss

In [150]:
def training_seq2seq(epochs, attention1, attention2):
  encoder1 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  encoder2 = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  decoder = DecoderWithDualAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention1, attention2)
  train_step_func = get_train_step_func()
  training_loss = []
  validation_loss = []

  for epoch in range(epochs):
    print()
    print(epoch)
    start = time.time()
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_loss = 0

    for (batch, (aux, inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      print(batch)
      batch_loss = train_step_func(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      # inp, targ, aux, enc_hidden, enc_hidden2, encoder, encoder2, decoder
      total_loss += batch_loss

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
        
    enc_hidden1 = encoder1.initialize_hidden_state()
    enc_hidden2 = encoder2.initialize_hidden_state()
    total_val_loss = 0
    for (batch, (aux, inp, targ)) in enumerate(validation_dataset.take(steps_per_epoch)):
      val_loss = caculate_validation_loss(inp, targ, aux, enc_hidden1, enc_hidden2, encoder1, encoder2, decoder)
      total_val_loss += val_loss

    training_loss.append(total_loss / steps_per_epoch)
    validation_loss.append(total_val_loss / steps_per_epoch_val)
    print('Epoch {} Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1,
                                        training_loss[-1], validation_loss[-1]))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
  return encoder1, encoder2, decoder, training_loss, validation_loss

In [88]:
# Training Starts Here

## Training seq2seq with Bahdanau attention

In [ ]:
epochs = 1

attention1 = BahdanauAttention(units)
attention2 = BahdanauAttention(units)
print("Running seq2seq model with Bahdanau attention")
encoder1_bah, encoder2_bah2, decoder_bah, training_loss, validation_loss = training_seq2seq(epochs, attention1, attention2)

# tloss = np.vstack((tloss, training_loss))
# vloss = np.vstack((vloss, validation_loss))

Running seq2seq model with Bahdanau attention

0
0


In [ ]:
def translate(g_context, r_context, encoder1, encoder2, decoder):
  attention_plot1 = np.zeros((max_length_targ, max_length_given))
  attention_plot2 = np.zeros((max_length_targ, max_length_reference))

  g_context = preprocess_sentence(g_context)

  given = [given_tokenizer.word_index[i] for i in g_context.split(' ')]
  given = tf.keras.preprocessing.sequence.pad_sequences([given],
                                                         maxlen=max_length_given,
                                                         padding='post')
  given = tf.convert_to_tensor(given)

  
  r_context = preprocess_sentence(r_context)
  
  ref = [reference_tokenizer.word_index[i] for i in r_context.split(' ')]
  ref = tf.keras.preprocessing.sequence.pad_sequences([ref],
                                                         maxlen=max_length_reference,
                                                         padding='post')
  ref = tf.convert_to_tensor(ref)


  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out1, enc_hidden1 = encoder1(given, hidden)
  enc_out2, enc_hidden2 = encoder2(ref, hidden)

  dec_hidden1 = enc_hidden1
  dec_hidden2 = enc_hidden2
  dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']], 0)

  for t in range(max_length_targ):
    # predictions, dec_hidden, attention_weights = decoder(dec_input,
    #                                                      dec_hidden,
    #                                                      enc_out)

    predictions, dec_hidden1, dec_hidden2, _, _ = decoder(dec_input, dec_hidden1, dec_hidden2, enc_out1, enc_out2)

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_tokenizer.index_word[predicted_id] + ' '

    # until the predicted word is <end>.
    if targ_tokenizer.index_word[predicted_id] == '<end>':
      return result

    # the predicted ID is fed back into the model, no teacher forcing.
    dec_input = tf.expand_dims([predicted_id], 0)

  return result

In [ ]:
translate("is my country. ","Its a really and beautiful place to live. ", encoder1_bah, encoder2_bah2, decoder_bah)

'what beyonce beyonce beyonce beyonce beyonce beyonce beyonce ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? '

### Doing it for a simple example batch

In [ ]:
# attention = BahdanauAttention(units)
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# decoder = DecoderWithAttention(vocab_tar_size, embedding_dim, units, BATCH_SIZE, attention)
# loss = 0
# with tf.GradientTape() as tape: # for automatic differentiation
#     sample_hidden = encoder.initialize_hidden_state()
#     enc_output, enc_hidden = encoder(example_input_batch, sample_hidden)

#     dec_hidden = enc_hidden
#     dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
#     # Teacher forcing - feeding the target as the next input
#     for t in range(1, example_target_batch.shape[1]):
#         # passing enc_output to the decoder
#         predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
#         loss += loss_function(example_target_batch[:, t], predictions)
#         # using teacher forcing
#         dec_input = tf.expand_dims(example_target_batch[:, t], 1)